# Clasificación por Random Forest


hacer el top 5 variables mas influyentes y en base a eso basicamente para ver si esto resplada lo que esta en el KNN

## Importar las librerías

In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler

## Importar el conjunto de datos

In [28]:
data = pd.read_csv('defunciones_clean.csv').round(0)
data = data[data.Causa_Defunción != 99]

X = data.drop(['Causa_Defunción'], axis=1)
y = data['Causa_Defunción']
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)

data = pd.concat([X_over, y_over], axis=1)

## Elimiinar las columnas que no se van a utilizar

In [29]:
# Eliminar columnas no relevantes (segun analisis exploratorio anteriormente realizado)
data.drop('Departamento_Registro', axis=1, inplace=True)
data.drop('Municipio_Registro', axis=1, inplace=True)

data.drop('Municipio_Ocurrencia', axis=1, inplace=True)   
data.drop('Departamento_Residencia_Madre', axis=1, inplace=True) 

data.drop('Municipio_Residencia_Madre', axis=1, inplace=True)
data.drop('Departamento_Ocurrencia', axis=1, inplace=True)

data.drop('Año_Ocurrencia', axis=1, inplace=True)
data.drop('Mes_Ocurrencia', axis=1, inplace=True)
data.drop('Día_Ocurrencia', axis=1, inplace=True)        # No se considera el día de ocurrencia

data.drop('Año_Registro', axis=1, inplace=True)
data.drop('Mes_Registro', axis=1, inplace=True)

data.head()

,Área_Geográfica,Asistencia_Recibida,Clase_Parto,Edad_Madre,Estado_Civil_Madre,Escolaridad_Madre,Ocupación_Madre,Semanas_Gestacion,Sexo,Sitio_Ocurrencia,Tipo_Atencion,Total_Hijos_Nacidos,Total_Hijos_Nacidos_Muertos,Total_Hijos_Vivos,Via_Parto,Nacionalidad_Madre,Grupo_Etnico_Madre,Causa_Defunción
0,99,6,1,18.0,2,99,99,34.0,1,2,1,1.0,2.0,1.0,99,1,4,0
1,99,1,1,36.0,2,99,1,34.0,1,9,1,1.0,3.0,2.0,99,1,4,0
2,99,2,1,22.0,2,99,99,34.0,1,2,1,1.0,3.0,2.0,99,1,4,0
3,99,2,1,26.0,2,99,99,34.0,1,2,1,1.0,0.0,0.0,99,1,4,0
4,99,2,1,43.0,2,99,99,34.0,2,2,1,1.0,10.0,6.0,99,1,4,0


## Crear el conjunto de entrenamiento y el conjunto de testing

In [30]:
X = data.drop('Causa_Defunción', axis=1)
y = data['Causa_Defunción']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

## Grid Search

In [41]:
param_grid = {
    'n_estimators': [20, 30, 40],
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

classifier = RandomForestClassifier(random_state=0)
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_val, y_val)

print(grid_search.best_params_)

{'criterion': 'gini', 'max_depth': 15, 'min_samples_split': 2, 'n_estimators': 40}


## Random Forest

Se crea el modelo de Random Forest con los parámetros obtenidos en el Grid Search. Asimismo se hace una serie de predicciones con el conjunto de testing para ver la precisión del modelo.

In [44]:
# Crear el modelo con los mejores parametros
classifier = RandomForestClassifier(criterion='gini', max_depth=15, min_samples_split=2, n_estimators=40)
classifier.fit(X_train, y_train)

# Predecir los valores del conjunto de validacion
y_pred = classifier.predict(X_test)

# Precision del Modelo
print("Precision:",precision_score(y_test, y_pred, average='weighted'))

# Recall del Modelo
print("Recall:",recall_score(y_test, y_pred, average='weighted'))

# F1 Score del Modelo
print("F1 Score",f1_score(y_test, y_pred, average='weighted'))

# Accuracy del Modelo
print("Accuracy:", classifier.score(X_test, y_test))


# Matriz de Confusion
confusion_matrix(y_test, y_pred)

Precision: 0.9348508662317597
Recall: 0.9281731405349211
F1 Score 0.9279704647025221
Accuracy: 0.9281731405349211


array([[4668,  705],
       [  55, 5153]])

Finalmente, se imprimen las 5 features del modelo que más importancia les dio el respectivo modelo.

In [35]:
importance_scores = classifier.feature_importances_
sorted_indices = importance_scores.argsort()[::-1]

top_five_indices = sorted_indices[:5]
top_five_variables = data.columns[top_five_indices]

print("Top five variables:\n", top_five_variables)
 

Top five variables:
 Index(['Semanas_Gestacion', 'Sitio_Ocurrencia', 'Edad_Madre',
       'Grupo_Etnico_Madre', 'Escolaridad_Madre'],
      dtype='object')
